In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [19]:
df_nivel_peru = pd.read_excel(r"C:\Users\gpavez\Desktop\Cramer Perú 2025.xlsx", sheet_name="cramer_peru")

In [24]:
df_nivel_peru.columns
df_nivel_peru_limpio = df_nivel_peru.drop({'Primer_Nivel', 'Segundo_Nivel', 'Centro_Costo', 'Departamento',
       'Nombre_Area', 'Nombre', "Edad", 'Jefatura'}, axis=1)
df_nivel_peru_limpio.head()

,DNI,Cargo,Antiguedad_Empresa,Sueldo_Base
0,7884700,GERENTE GENERAL,25.2,50990
1,10554111,SUB GERENTE VENTA SABORES,22.6,30000
2,40821076,JEFE COMERCIAL UNIDAD CÁRNICA,10.2,22500
3,41532675,JEFE TÉCNICO COMERCIAL DIVISIÓN FRAGANCIAS,16.4,20000
4,44392341,KEY ACCOUNT MANAGER,10.1,17100


In [10]:
# Función para crear análisis de rangos salariales por nivel
def crear_analisis_rangos_salariales(df, columna_grado, columna_sueldo, fuente):
    # Filtrar registros que tengan tanto grado como sueldo
    df_filtrado = df[(df[columna_grado].notna()) & (df[columna_sueldo].notna())].copy()
    
    if df_filtrado.empty:
        return pd.DataFrame(columns=['Fuente', 'Nivel', 'Cantidad', 'Sueldo_Minimo', 'Sueldo_Maximo', 'Punto_Medio', 'Mediana'])
    
    # Convertir sueldos a numérico
    df_filtrado[columna_sueldo] = pd.to_numeric(df_filtrado[columna_sueldo], errors='coerce')
    
    # Agrupar por nivel y calcular estadísticas
    resultado = []
    
    for nivel in df_filtrado[columna_grado].unique():
        df_nivel = df_filtrado[df_filtrado[columna_grado] == nivel]
        sueldos = df_nivel[columna_sueldo].dropna()
        
        if len(sueldos) > 0:
            minimo = sueldos.min()
            maximo = sueldos.max()
            punto_medio = (minimo + maximo) / 2
            mediana = sueldos.median()
            cantidad = len(sueldos)
            
            resultado.append({
                'Fuente': fuente,
                'Nivel': nivel,
                'Cantidad': cantidad,
                'Sueldo_Minimo': minimo,
                'Sueldo_Maximo': maximo,
                'Punto_Medio': punto_medio,
                'Mediana': mediana
            })
    
    # Crear DataFrame y ordenar por nivel
    df_resultado = pd.DataFrame(resultado)
    
    # Función para ordenar niveles (números primero, luego romanos)
    def ordenar_nivel(nivel):
        try:
            return (0, int(float(nivel)))  # Números normales
        except:
            # Intentar convertir romano a número
            romano_convertido = romano_a_entero(nivel)
            if romano_convertido is not None:
                return (1, romano_convertido)
            else:
                return (2, str(nivel))  # Otros valores como string
    
    if not df_resultado.empty:
        df_resultado['orden'] = df_resultado['Nivel'].apply(ordenar_nivel)
        df_resultado = df_resultado.sort_values('orden').drop('orden', axis=1)
    
    return df_resultado

# Crear análisis para Carlos (usando datos de df_unificado)
print("Creando análisis de rangos salariales...")

# Para Carlos - usar columna de sueldo de Carlos si existe, sino usar la de Deborah
if 'Sueldo Base_carlos' in df_unificado.columns:
    rangos_carlos = crear_analisis_rangos_salariales(df_unificado, 'grado_carlos', 'Sueldo Base_carlos', 'Carlos')
else:
    # Si no hay columna específica de Carlos, usar la general
    rangos_carlos = crear_analisis_rangos_salariales(df_unificado, 'grado_carlos', 'Sueldo Base', 'Carlos')

# Para Deborah
if 'Sueldo Base_deborah' in df_unificado.columns:
    rangos_deborah = crear_analisis_rangos_salariales(df_unificado, 'grado_deborah', 'Sueldo Base_deborah', 'Deborah')
else:
    rangos_deborah = crear_analisis_rangos_salariales(df_unificado, 'grado_deborah', 'Sueldo Base', 'Deborah')

# Combinar ambos análisis en un solo DataFrame
rangos_combinados = pd.concat([rangos_carlos, rangos_deborah], ignore_index=True)

# Mostrar resultados
print(f"\nAnálisis Carlos - {len(rangos_carlos)} niveles encontrados:")
if not rangos_carlos.empty:
    print(rangos_carlos.head())

print(f"\nAnálisis Deborah - {len(rangos_deborah)} niveles encontrados:")
if not rangos_deborah.empty:
    print(rangos_deborah.head())

# Formatear números para mejor visualización
def formatear_moneda(df):
    columnas_dinero = ['Sueldo_Minimo', 'Sueldo_Maximo', 'Punto_Medio', 'Mediana']
    df_formateado = df.copy()
    for col in columnas_dinero:
        if col in df_formateado.columns:
            df_formateado[col] = df_formateado[col].round(0).astype(int)
    return df_formateado

rangos_carlos_formato = formatear_moneda(rangos_carlos)
rangos_deborah_formato = formatear_moneda(rangos_deborah)
rangos_combinados_formato = formatear_moneda(rangos_combinados)

print("\nDatos listos para exportar a Excel...")

Creando análisis de rangos salariales...

Análisis Carlos - 9 niveles encontrados:
   Fuente  Nivel  Cantidad  Sueldo_Minimo  Sueldo_Maximo  Punto_Medio  \
5  Carlos   10.0        34       550000.0      1102000.0     826000.0   
3  Carlos   11.0       155       540000.0      1580000.0    1060000.0   
8  Carlos   12.0        11       992000.0      1230000.0    1111000.0   
6  Carlos   13.0        88       610000.0      1614000.0    1112000.0   
7  Carlos   14.0        68       698000.0      3300000.0    1999000.0   

     Mediana  
5   626000.0  
3   690000.0  
8  1070000.0  
6   781500.0  
7  1300000.0  

Análisis Deborah - 17 niveles encontrados:
     Fuente Nivel  Cantidad  Sueldo_Minimo  Sueldo_Maximo  Punto_Medio  \
5   Deborah     1         6       540000.0       700000.0     620000.0   
11  Deborah     2        23       570000.0       660000.0     615000.0   
13  Deborah     3        20       570000.0       680000.0     625000.0   
4   Deborah     4       108       610000.0      

In [12]:
# Exportar todos los DataFrames a Excel con múltiples hojas
with pd.ExcelWriter(r"C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\niveles_unificados1.2.xlsx", 
                    engine='openpyxl') as writer:
    
    # Hoja 1: Datos unificados (datos originales)
    df_unificado_ordenado.to_excel(writer, 
                                   sheet_name='Niveles_Unificados', 
                                   index=False)
    
    # Hoja 2: Resumen de niveles por cargo
    df_niveles_separados.to_excel(writer, 
                                  sheet_name='Resumen_Niveles_Por_Cargo', 
                                  index=False)
    
    # Hoja 3: Rangos salariales Carlos
    rangos_carlos_formato.to_excel(writer,
                                   sheet_name='Rangos_Carlos',
                                   index=False)
    
    # Hoja 4: Rangos salariales Deborah  
    rangos_deborah_formato.to_excel(writer,
                                    sheet_name='Rangos_Deborah',
                                    index=False)
    
    # Hoja 5: Rangos combinados
    rangos_combinados_formato.to_excel(writer,
                                       sheet_name='Rangos_Combinados',
                                       index=False)

print("Archivo Excel exportado exitosamente!")
print(f"Ubicación: C:\\Users\\gpavez\\Desktop\\Compensaciones\\git\\compensaciones\\niveles_unificados1.2.xlsx")
print(f"Hoja 1 - Niveles_Unificados: {len(df_unificado_ordenado)} registros")
print(f"Hoja 2 - Resumen_Niveles_Por_Cargo: {len(df_niveles_separados)} registros")
print(f"Hoja 3 - Rangos_Carlos: {len(rangos_carlos_formato)} niveles")
print(f"Hoja 4 - Rangos_Deborah: {len(rangos_deborah_formato)} niveles")
print(f"Hoja 5 - Rangos_Combinados: {len(rangos_combinados_formato)} registros")
print("Hojas creadas:")
print("  - Niveles_Unificados: Datos detallados por persona")
print("  - Resumen_Niveles_Por_Cargo: Niveles únicos agrupados por cargo")
print("  - Rangos_Carlos: Análisis salarial por nivel (Carlos)")
print("  - Rangos_Deborah: Análisis salarial por nivel (Deborah)")
print("  - Rangos_Combinados: Comparación de ambas fuentes")

Archivo Excel exportado exitosamente!
Ubicación: C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\niveles_unificados1.2.xlsx
Hoja 1 - Niveles_Unificados: 574 registros
Hoja 2 - Resumen_Niveles_Por_Cargo: 157 registros
Hoja 3 - Rangos_Carlos: 9 niveles
Hoja 4 - Rangos_Deborah: 17 niveles
Hoja 5 - Rangos_Combinados: 26 registros
Hojas creadas:
  - Niveles_Unificados: Datos detallados por persona
  - Resumen_Niveles_Por_Cargo: Niveles únicos agrupados por cargo
  - Rangos_Carlos: Análisis salarial por nivel (Carlos)
  - Rangos_Deborah: Análisis salarial por nivel (Deborah)
  - Rangos_Combinados: Comparación de ambas fuentes


In [25]:
# Función para crear análisis de rangos salariales por cargo
def crear_analisis_rangos_por_cargo(df, columna_cargo, columna_sueldo, fuente):
    # Filtrar registros que tengan tanto cargo como sueldo
    df_filtrado = df[(df[columna_cargo].notna()) & (df[columna_sueldo].notna())].copy()
    
    if df_filtrado.empty:
        return pd.DataFrame(columns=['Fuente', 'Cargo', 'Cantidad', 'Sueldo_Minimo', 'Sueldo_Maximo', 'Promedio', 'Mediana'])
    
    # Convertir sueldos a numérico
    df_filtrado[columna_sueldo] = pd.to_numeric(df_filtrado[columna_sueldo], errors='coerce')
    
    # Agrupar por cargo y calcular estadísticas
    resultado = []
    
    for cargo in df_filtrado[columna_cargo].unique():
        df_cargo = df_filtrado[df_filtrado[columna_cargo] == cargo]
        sueldos = df_cargo[columna_sueldo].dropna()
        
        if len(sueldos) > 0:
            minimo = sueldos.min()
            maximo = sueldos.max()
            promedio = sueldos.mean()
            mediana = sueldos.median()
            cantidad = len(sueldos)
            
            resultado.append({
                'Fuente': fuente,
                'Cargo': cargo,
                'Cantidad': cantidad,
                'Sueldo_Minimo': minimo,
                'Sueldo_Maximo': maximo,
                'Promedio': promedio,
                'Mediana': mediana
            })
    
    # Crear DataFrame y ordenar por cargo
    df_resultado = pd.DataFrame(resultado)
    
    if not df_resultado.empty:
        df_resultado = df_resultado.sort_values('Cargo')
    
    return df_resultado

# Análisis de rangos salariales por cargo para Perú
print("Creando análisis de rangos salariales por cargo para Perú...")

# Verificar qué columnas están disponibles
print("Columnas disponibles en df_nivel_peru_limpio:")
print(df_nivel_peru_limpio.columns.tolist())

# Crear análisis por cargo - ajustar nombres de columnas según tu DataFrame
# Asumiendo que existe una columna de cargo y una de sueldo
if 'Cargo' in df_nivel_peru_limpio.columns and 'Sueldo_Base' in df_nivel_peru_limpio.columns:
    rangos_cargo_peru = crear_analisis_rangos_por_cargo(df_nivel_peru_limpio, 'Cargo', 'Sueldo_Base', 'Perú')
elif 'cargo' in df_nivel_peru_limpio.columns and 'sueldo' in df_nivel_peru_limpio.columns:
    rangos_cargo_peru = crear_analisis_rangos_por_cargo(df_nivel_peru_limpio, 'cargo', 'sueldo', 'Perú')
else:
    # Mostrar columnas para identificar los nombres correctos
    print("Por favor, identifica las columnas correctas:")
    print(df_nivel_peru_limpio.head())
    rangos_cargo_peru = pd.DataFrame()

if not rangos_cargo_peru.empty:
    print(f"\nAnálisis por cargo - {len(rangos_cargo_peru)} cargos encontrados:")
    
    # Formatear números para mejor visualización
    rangos_cargo_peru_formato = rangos_cargo_peru.copy()
    columnas_dinero = ['Sueldo_Minimo', 'Sueldo_Maximo', 'Promedio', 'Mediana']
    for col in columnas_dinero:
        if col in rangos_cargo_peru_formato.columns:
            rangos_cargo_peru_formato[col] = rangos_cargo_peru_formato[col].round(0).astype(int)
    
    print(rangos_cargo_peru_formato)
else:
    print("No se pudo crear el análisis. Verifica los nombres de las columnas.")

Creando análisis de rangos salariales por cargo para Perú...
Columnas disponibles en df_nivel_peru_limpio:
['DNI', 'Cargo', 'Antiguedad_Empresa', 'Sueldo_Base']

Análisis por cargo - 46 cargos encontrados:
   Fuente                                              Cargo  Cantidad  \
32   Perú                                ANALISTA DE CALIDAD         2   
22   Perú        ANALISTA DE EVALUACION SENSORIAL FRAGANCIAS         1   
34   Perú                           ASISTENTE ADMINISTRATIVO         1   
35   Perú     ASISTENTE ADMINISTRATIVO Y SERVICIO AL CLIENTE         1   
27   Perú                                 ASISTENTE CONTABLE         2   
26   Perú  ASISTENTE DE ADMINISTRACIÓN Y SERVICIO AL CLIENTE         2   
31   Perú  ASISTENTE DE APLICACIONES SABORES DULCES Y BEB...         1   
43   Perú                              ASISTENTE DE MUESTRAS         3   
41   Perú    ASISTENTE DE REENVASADO Y APLICACIONES CARNICAS         1   
30   Perú     ASISTENTE DE SERVICIO AL CLIENTE DE FRAG

In [26]:
# Exportar análisis por cargo a Excel (si existe)
if not rangos_cargo_peru.empty:
    # Formatear datos
    rangos_cargo_peru_formato = rangos_cargo_peru.copy()
    columnas_dinero = ['Sueldo_Minimo', 'Sueldo_Maximo', 'Promedio', 'Mediana']
    for col in columnas_dinero:
        if col in rangos_cargo_peru_formato.columns:
            rangos_cargo_peru_formato[col] = rangos_cargo_peru_formato[col].round(0).astype(int)
    
    # Exportar a Excel
    with pd.ExcelWriter(r"C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\analisis_cargos_peru.xlsx", 
                        engine='openpyxl') as writer:
        
        # Hoja 1: Datos originales
        df_nivel_peru_limpio.to_excel(writer, 
                                      sheet_name='Datos_Originales', 
                                      index=False)
        
        # Hoja 2: Análisis por cargo
        rangos_cargo_peru_formato.to_excel(writer,
                                           sheet_name='Analisis_Por_Cargo',
                                           index=False)
    
    print("Archivo Excel exportado exitosamente!")
    print(f"Ubicación: C:\\Users\\gpavez\\Desktop\\Compensaciones\\git\\compensaciones\\analisis_cargos_peru.xlsx")
    print(f"Hoja 1 - Datos_Originales: {len(df_nivel_peru_limpio)} registros")
    print(f"Hoja 2 - Analisis_Por_Cargo: {len(rangos_cargo_peru_formato)} cargos")
else:
    print("No se exportó archivo porque no hay datos para analizar.")

Archivo Excel exportado exitosamente!
Ubicación: C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\analisis_cargos_peru.xlsx
Hoja 1 - Datos_Originales: 71 registros
Hoja 2 - Analisis_Por_Cargo: 46 cargos
